In [1]:
#导入库并产生数据
import math
import numpy as np
import torch
from torch.utils import data
import random

#定义正态分布
def normal(x,mu,sigma):
    p=1/math.sqrt(2*math.pi*sigma*sigma)
    return p*np.exp(-0.5/sigma**2 *(x-mu)**2)

#制造数据集 y=Xw+b+噪声
def synthetic_data(w, b, num_examples): 
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

#"""构造⼀个PyTorch数据迭代器"""
def load_array(data_arrays, batch_size, is_train=True): 
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


In [2]:
true_w, true_b = torch.ones((200, 1)) * 0.01, 0.05
train_data = synthetic_data(true_w, true_b, 20)
train_iter = load_array(train_data, 5)
test_data = synthetic_data(true_w, true_b, 100)
test_iter = load_array(test_data, 5, is_train=False)

在pytorch神经网络中已经为我们设置好了超参数，我们可通过改变超参数调整惩罚系数

In [3]:
from torch import nn
net=nn.Sequential(nn.Linear(20,1))


num_epoch=100
def set_sgd(wd):
    sgd=torch.optim.SGD([{"params":net[0].weight,'weight_decay': wd},
{"params":net[0].bias}], lr=0.1)
    return sgd


In [ ]:
def train_tr(wd,num_epoch,net,train_data,train_iter):
    net[0].weight.data.normal_(0, 0.01)
    net[0].bias.data.fill_(0)
    loss=nn.MSELoss()
    sgd=set_sgd(wd)
    features,labels=train_data
    
    for epoch in range(num_epoch):
        for X,y in train_iter:
            l=loss(net(X),y)
            sgd.zero_grad()
            l.backforward()
            sgd.step()
        l=loss(net(features), labels)

